# Production-Ready Tool Design Patterns

This notebook demonstrates:
- **Text Analyzer**: Structured JSON output with single responsibility
- **Calculator**: Input validation and safe computation with try/except
- **Weather API**: External data retrieval with error handling and API integration

In [1]:
from strands import Agent, tool
from strands.models import BedrockModel
from typing import Dict, Any
import json
import requests

## Tool 1: Text Analyzer

**Pattern**: Structured JSON output with single responsibility

In [2]:
@tool
def analyze_text(text: str) -> str:
    """
    Analyze text and return structured metrics.
    
    Args:
        text: Text to analyze
    
    Returns:
        JSON string with word_count, char_count, sentence_count, avg_word_length
    """
    try:
        # Single responsibility: text analysis only
        words = text.split()
        sentences = text.count('.') + text.count('!') + text.count('?')
        
        result = {
            "word_count": len(words),
            "char_count": len(text),
            "sentence_count": max(sentences, 1),
            "avg_word_length": round(sum(len(w) for w in words) / len(words), 2) if words else 0
        }
        
        # Return structured JSON
        return json.dumps(result, indent=2)
    
    except Exception as e:
        return json.dumps({"error": str(e)})

In [3]:
# Test text analyzer
sample_text = "The quick brown fox jumps over the lazy dog. This is a test sentence!"
result = analyze_text(sample_text)
print("Text Analysis Result:")
print(result)

Text Analysis Result:
{
  "word_count": 14,
  "char_count": 69,
  "sentence_count": 2,
  "avg_word_length": 4.0
}


## Tool 2: Calculator

**Pattern**: Input validation and safe computation with try/except

In [4]:
@tool
def calculate(expression: str) -> str:
    """
    Safely evaluate mathematical expressions.
    
    Args:
        expression: Math expression (e.g., "2 + 2", "10 * 5 / 2")
    
    Returns:
        JSON string with result or error
    """
    try:
        # Input validation: only allow safe characters
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return json.dumps({
                "error": "Invalid characters in expression",
                "expression": expression
            })
        
        # Safe computation with restricted eval
        result = eval(expression, {"__builtins__": {}}, {})
        
        return json.dumps({
            "expression": expression,
            "result": result
        }, indent=2)
    
    except ZeroDivisionError:
        return json.dumps({
            "error": "Division by zero",
            "expression": expression
        })
    
    except SyntaxError:
        return json.dumps({
            "error": "Invalid syntax",
            "expression": expression
        })
    
    except Exception as e:
        return json.dumps({
            "error": f"Calculation error: {str(e)}",
            "expression": expression
        })

In [5]:
# Test calculator with valid expressions
print("Valid calculation:")
print(calculate("(10 + 5) * 2"))

print("\nDivision by zero:")
print(calculate("10 / 0"))

print("\nInvalid characters:")
print(calculate("import os"))

Valid calculation:
{
  "expression": "(10 + 5) * 2",
  "result": 30
}

Division by zero:
{"error": "Division by zero", "expression": "10 / 0"}

Invalid characters:
{"error": "Invalid characters in expression", "expression": "import os"}


## Tool 3: Weather API

**Pattern**: External data retrieval with error handling and API integration

In [21]:
@tool
def get_weather(city: str) -> str:
    """
    Get weather information for a city.
    
    Args:
        city: City name
    
    Returns:
        JSON string with weather data or error
    """
    try:
        # Input validation
        if not city or not city.strip():
            return json.dumps({"error": "City name cannot be empty"})
        
        # API integration with timeout
        # Using wttr.in as a free weather API (no key required)
        url = f"https://wttr.in/{city}?format=j1"
        
        response = requests.get(url, timeout=6)
        
        # HTTP error handling
        if response.status_code == 404:
            return json.dumps({
                "error": "City not found",
                "city": city
            })
        
        response.raise_for_status()
        
        # Parse and structure response
        data = response.json()
        current = data['current_condition'][0]
        
        result = {
            "city": city,
            "temperature_f": current['temp_F'],
            "temperature_c": current['temp_C'],
            "condition": current['weatherDesc'][0]['value'],
            "humidity": current['humidity'],
            "wind_speed_mph": current['windspeedMiles']
        }
        
        return json.dumps(result, indent=2)
    
    except requests.Timeout:
        return json.dumps({
            "error": "Request timeout - API took too long to respond",
            "city": city
        })
    
    except requests.ConnectionError:
        return json.dumps({
            "error": "Connection error - check internet connection",
            "city": city
        })
    
    except requests.HTTPError as e:
        return json.dumps({
            "error": f"HTTP error: {e.response.status_code}",
            "city": city
        })
    
    except KeyError as e:
        return json.dumps({
            "error": f"Unexpected API response format: missing {str(e)}",
            "city": city
        })
    
    except Exception as e:
        return json.dumps({
            "error": f"Unexpected error: {str(e)}",
            "city": city
        })

In [23]:
# Test weather API
print("Weather for Vancouver, BC:")
print(get_weather("yvr"))

Weather for Vancouver, BC:
{
  "city": "yvr",
  "temperature_f": "52",
  "temperature_c": "11",
  "condition": "Light rain",
  "humidity": "100",
  "wind_speed_mph": "5"
}


## Create Agent with All Tools

In [25]:
agent = Agent(
    system_prompt="""You are a helpful assistant with access to:
    - Text analysis for content metrics
    - Calculator for mathematical computations
    - Weather API for current weather information
    
    Use the appropriate tool based on the user's request.""",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[analyze_text, calculate, get_weather]
)

## Test Agent with Different Tool Invocations

In [26]:
# Text analysis request
response = agent("Analyze this text: 'AI agents are transforming software development.'")
print("=== Text Analysis ===")
print(response)

<thinking> The user has requested to analyze a given text and obtain metrics such as word count, character count, sentence count, and average word length. The appropriate tool for this task is "analyze_text". I will use this tool to analyze the provided text.</thinking>

Tool #1: analyze_text
Here are the analysis results for the provided text "AI agents are transforming software development":

- Word count: 6
- Character count: 48
- Sentence count: 1
- Average word length: 7.17 characters

If you need any further assistance or analysis, feel free to ask!=== Text Analysis ===
Here are the analysis results for the provided text "AI agents are transforming software development":

- Word count: 6
- Character count: 48
- Sentence count: 1
- Average word length: 7.17 characters

If you need any further assistance or analysis, feel free to ask!



In [27]:
# Calculation request
response = agent("Calculate (25 * 4) + (100 / 5)")
print("=== Calculation ===")
print(response)

<thinking> To calculate the expression (25 * 4) + (100 / 5), I will first perform the multiplication and division as per the order of operations (PEMDAS/BODMAS), and then add the results together.</thinking>

Tool #2: calculate

Tool #3: calculate
The results from the calculations are as follows:

- The result of (25 * 4) is 100.
- The result of (100 / 5) is 20.

Now, I will add these two results together:

100 + 20 = 120

Therefore, the final result of the expression (25 * 4) + (100 / 5) is **120**.=== Calculation ===
The results from the calculations are as follows:

- The result of (25 * 4) is 100.
- The result of (100 / 5) is 20.

Now, I will add these two results together:

100 + 20 = 120

Therefore, the final result of the expression (25 * 4) + (100 / 5) is **120**.



In [28]:
# Weather request
response = agent("What's the weather like in New York?")
print("=== Weather Query ===")
print(response)

<thinking> To provide the current weather information for New York, I will use the "get_weather" tool. This tool will retrieve the latest weather data for the specified city.</thinking> 
Tool #4: get_weather
I encountered an error while trying to retrieve the weather information for New York. It seems there was a connection issue. 

To resolve this, please make sure your internet connection is stable, and try again if you need the weather information. If you have any other requests or need assistance with a different topic, feel free to let me know!=== Weather Query ===
I encountered an error while trying to retrieve the weather information for New York. It seems there was a connection issue. 

To resolve this, please make sure your internet connection is stable, and try again if you need the weather information. If you have any other requests or need assistance with a different topic, feel free to let me know!



In [29]:
# Multi-tool request
response = agent("""
Analyze this sentence: 'The weather in San Francisco is beautiful today.'
Then get the actual weather for San Francisco.
""")
print("=== Multi-Tool Request ===")
print(response)

<thinking> First, I will analyze the given sentence using the "analyze_text" tool to get metrics such as word count, character count, sentence count, and average word length. After that, I will use the "get_weather" tool to retrieve the actual weather information for San Francisco.</thinking>


Tool #5: analyze_text

Tool #6: get_weather
Here are the analysis results for the provided sentence "The weather in San Francisco is beautiful today":

- Word count: 8
- Character count: 48
- Sentence count: 1
- Average word length: 5.12 characters

And here is the current weather information for San Francisco:

- Temperature: 54°F (12°C)
- Condition: Clear
- Humidity: 80%
- Wind speed: 7 mph

If you need any further information or have any other requests, feel free to ask!=== Multi-Tool Request ===
Here are the analysis results for the provided sentence "The weather in San Francisco is beautiful today":

- Word count: 8
- Character count: 48
- Sentence count: 1
- Average word length: 5.12 chara

## Best Practices Demonstrated

### Text Analyzer
✅ Single responsibility (only text analysis)  
✅ Structured JSON output  
✅ Consistent return format  

### Calculator
✅ Input validation (character whitelist)  
✅ Safe computation (restricted eval)  
✅ Specific exception handling (ZeroDivisionError, SyntaxError)  
✅ Informative error messages  

### Weather API
✅ Input validation (empty check)  
✅ Timeout configuration  
✅ HTTP status code handling  
✅ Multiple exception types (Timeout, ConnectionError, HTTPError)  
✅ Graceful degradation with error context  